# test

In [ ]:
import math
import numpy as np
import collections


def handle_struct_entropy(subject_dict):
    """结构熵"""

    in_data = []
    out_data = []
    all_data = []
    
    for obj in subject_dict.values():
        
        if obj.get('i'):
            in_data.append(len(obj['i']))
            
        if obj.get('o'):
            out_data.append(len(obj['o']))
        
        cache = len(obj.get('i',set())) + len(obj.get('o',set()))
        if cache:
            all_data.append(cache)
            
    np_in = np.array(in_data)
    np_out = np.array(out_data)
    np_all = np.array(all_data)
    
#     print(np_in,np_out,np_all)
    
    f = np_in/np_in.sum()
    a = (- f * np.log2(f)).sum()
    
    f = np_out/np_out.sum()
    b = (- f * np.log2(f)).sum()
    
    f = np_all/np_all.sum()
    c = (- f * np.log2(f)).sum()
    
    # E min
    i_E_min = math.log2(4 * (len(np_in)-1))/2
    o_E_min = math.log2(4 * (len(np_out)-1))/2
    all_E_min = math.log2(4 * (len(np_all)-1))/2
    
    retin = (a - i_E_min) / ( math.log2(len(np_in)) - i_E_min)
    retout = (b - o_E_min) / ( math.log2(len(np_out)) - o_E_min)
    # retin = a /  math.log2(len(np_all))
    # retout = b /  math.log2(len(np_all))
    retall = (c - all_E_min) / (math.log2(len(np_all)) - all_E_min)

    return retin,retout,retall,a,b,c,len(np_in),len(np_out),len(np_all)

def handle_degree_distrubution_entropy(subject_dict):
    '''度分布熵'''
    
    in_data = collections.defaultdict(int)
    out_data = collections.defaultdict(int)
    all_data = collections.defaultdict(int)
    
    for i,obj in subject_dict.items():
        
        in_data[len(obj['i'])] += 1
            
        out_data[len(obj['o'])] += 1
        
        cache = len(obj.get('i',set())) + len(obj.get('o',set()))
        if cache > 0 :
            all_data[cache] += 1
             
    np_in = np.array(list(in_data.values()))
    np_out = np.array(list(out_data.values()))
    np_all = np.array(list(all_data.values()))
    
    np_in = np_in / np_in.sum()
    np_out = np_out / np_out.sum()
    np_all = np_all / np_all.sum()
    
    ret_in = np.nansum( np_in *  np.log2( 1/np_in ) )
    ret_out = np.nansum( np_out *  np.log2( 1/np_out ) )
    ret_all = np.nansum( np_all *  np.log2( 1/np_all ) )
    
    
    return {
        'i':ret_in,
        'o':ret_out,
        'a':ret_all,
    }

def handle_entropy_value(edges):
    
    subject_dict = {}
    for (node_in,node_out) in edges:
        subject_dict.setdefault(node_in,{
            'i': [],
            'o':[]
        })
        subject_dict.setdefault(node_out,{
            'i':[],
            'o':[]
        })
        subject_dict[node_in]['o'].append(node_out)
        subject_dict[node_out]['i'].append(node_in)
        
        
    data0 = handle_degree_distrubution_entropy(subject_dict)
    
    data1 = handle_struct_entropy(subject_dict)
    return (data0['i'],data0['o'], data0['a']),(data1[0],data1[1],data1[2],data1[3],data1[4],data1[5])


In [6]:

handle_entropy_value([(1,2),(2,3),(3,4),(4,5),(5,1)])

((np.float64(0.0), np.float64(0.0), np.float64(0.0)),
 (np.float64(1.0),
  np.float64(1.0),
  np.float64(1.0),
  np.float64(2.321928094887362),
  np.float64(2.321928094887362),
  np.float64(2.321928094887362)))

In [15]:

# handle_entropy_value([(1,2),(1,3),(1,4),(1,5),(6,1)])
handle_entropy_value([(1,2),(1,3),(4,1),(2,3),(4,2),(4,3)])

((np.float64(2.0), np.float64(2.0), np.float64(0.0)),
 (np.float64(-0.4808248653936201),
  np.float64(-0.4808248653936201),
  np.float64(1.0),
  np.float64(1.4591479170272448),
  np.float64(1.4591479170272448),
  np.float64(2.0)))

In [17]:
# !pip install networkx

In [18]:
# import graph_tool.all as gt

In [27]:
import networkx as nx
import matplotlib.pyplot as plt

# 设置网络中的节点数和每个新节点连接的节点数
# n = 9  # 节点数
# m = 3    # 每个新节点连接的节点数
# BA_graph = nx.barabasi_albert_graph(n, m)

# 将无向图转换为有向图
# BA_directed = BA_graph.to_directed()

# 可视化有向网络
# pos = nx.spring_layout(BA_directed)  # 为有向图设置布局
# nx.draw(BA_directed, pos, with_labels=False, node_size=30, arrowstyle='-|>', arrowsize=10)
# plt.show()

In [2]:
# test

edges = [
    (0, 1), (1, 2), (2, 1), (3, 4), (4, 5), (6, 4), (7, 4)
]
handle_entropy_value(edges)

((np.float64(1.75),
  np.float64(0.5435644431995963),
  np.float64(1.5487949406953985)),
 (np.float64(0.24041077205417438),
  np.float64(1.0),
  np.float64(0.586523068142618),
  np.float64(1.8423709931771084),
  np.float64(2.807354922057604),
  np.float64(2.753434386188785)))

In [98]:
import random
def generate_directed_BA(n, m):
    # 初始化有向图，并添加初始的 m+1 个节点
    G = nx.DiGraph()
    for i in range(m + 1):
        G.add_node(i)
    
    # 手动为初始节点添加边，确保它们之间有 m 条边
    for i in range(m + 1):
        targets = random.sample([node for node in G.nodes if node != i], m)
        for target in targets:
            G.add_edge(i, target)
    
    # 继续添加节点，每个新节点添加 m 条边，遵循优先连接机制
    for new_node in range(m + 1, n):
        G.add_node(new_node)
        targets = random.choices(list(G.nodes), weights=[G.in_degree(node) + 1 for node in G.nodes], k=m)
        for target in targets:
            G.add_edge(new_node, target)
    
    return G

In [109]:

n = 500  # 节点数
m = 2    # 每个新节点连接的节点数
BA_graph = generate_directed_BA(n, m)
handle_entropy_value(BA_graph.edges())

((np.float64(2.1113354271948586),
  np.float64(0.18426059333965505),
  np.float64(2.1723757482335913)),
 (np.float64(0.5207725152008894),
  np.float64(0.9982373500071005),
  np.float64(0.7713732561824773),
  np.float64(6.295257905942307),
  np.float64(8.959642619339371),
  np.float64(8.16917182614467)))